<a href="https://colab.research.google.com/github/LauraSchellekens/TM10007_-private_-group/blob/Laura_random_forest/Random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
# toepassen random forest op onze data 

# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/karinvangarderen/tm10007_project.git
!pip install sklearn numpy matplotlib


In [0]:
# General packages
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import impute
from sklearn import preprocessing

In [43]:
# Data loading functions. 
from adni.load_data import load_data
data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')
X = data.drop('label', axis=1) # All data without column 'label'
Y = data['label'] # AD en CN moeten alleen nog omgezet worden naar 0 en 1
Y.replace(('AD', 'CN'), (0, 1), inplace=True) # AD en CN omgezet naar 0 en 1
print("number of AD patients:")
print((list(data['label'] == 0)).count(True)) # displays number AD patients
print("number of CN patients:")
print((list(data['label'] == 0)).count(False)) # displays number of CN patients 

ERROR! Session/line number was not unique in database. History logging moved to new session 59
The number of samples: 855
The number of columns: 268
number of AD patients:
519
number of CN patients:
336


In [35]:
# split into train (70%) and test (30%) set

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(598, 267) (598,)
(257, 267) (257,)


In [36]:
# Preprocessing: drop feature if too many missing values

X_train = X_train.replace(0, np.NaN)                  # replace 0 with NaN
missing_per_feature = X_train.isnull().sum()          # gives the amount of missing values (NaN) per feature
pct_null = missing_per_feature / len(X_train)         # gives percentage of missing values per feature
missing_features = pct_null[pct_null > 0.40].index    # gives features with more than 40% missing values
X_train.drop(missing_features, axis=1, inplace=True)  # remove feature if more than 40% missing values

missing_per_feature_max = max(X_train.isnull().sum()) # gives the maximum amount of missing values (NaN) per feature AFTER dropping incomplete features

print(missing_per_feature_max)
print (X_train.shape, y_train.shape)

97
(598, 261) (598,)


In [37]:
# nu ook voor X_test
X_test = X_test.replace(0, np.NaN)                  # replace 0 with NaN
missing_per_feature = X_test.isnull().sum()          # gives the amount of missing values (NaN) per feature
pct_null = missing_per_feature / len(X_test)         # gives percentage of missing values per feature
missing_features = pct_null[pct_null > 0.40].index    # gives features with more than 40% missing values
X_test.drop(missing_features, axis=1, inplace=True)  # remove feature if more than 40% missing values

missing_per_feature_max = max(X_test.isnull().sum()) # gives the maximum amount of missing values (NaN) per feature AFTER dropping incomplete features

print(missing_per_feature_max)
print (X_test.shape, y_test.shape)

43
(257, 261) (257,)


In [38]:
# Preprocessing:  impute missing values (fill missing)

imputer = impute.SimpleImputer(strategy='mean')     # imputer with mean 
X_train_imp = imputer.fit_transform(X_train)          # impute  
X_train_imp = pd.DataFrame(data=X_train_imp, index=[X_train.index], columns=[X_train.columns]) # turn created np.array back to pandas df

type(X_train_imp)

# en nu ook voor x_test
imputer = impute.SimpleImputer(strategy='mean')     # imputer with mean 
X_test_imp = imputer.fit_transform(X_test)          # impute  
X_test_imp = pd.DataFrame(data=X_test_imp, index=[X_test.index], columns=[X_test.columns]) # turn created np.array back to pandas df

type(X_test_imp)

pandas.core.frame.DataFrame

In [0]:
# Preprocessing: scaling either standard or robust (removes median and scales data according to quantile range) 

scaler = preprocessing.StandardScaler()
scaler.fit_transform(X_train_imp)
X_train_scaled = scaler.transform(X_train_imp)
# robust = preprocessing.RobustScaler()
# robust.fit_transform(X_train_imp)
# X_train_scaled = robust.transform(X_train_imp)

X_train_scaled = pd.DataFrame(data=X_train_scaled, index=[X_train.index], columns=[X_train.columns]) # turn created np.array back to pandas df

# nu ook voor X_TEST
scaler = preprocessing.StandardScaler()
scaler.fit_transform(X_test_imp)
X_test_scaled = scaler.transform(X_test_imp)
# robust = preprocessing.RobustScaler()
# robust.fit_transform(X_train_imp)
# X_train_scaled = robust.transform(X_train_imp)

X_test_scaled = pd.DataFrame(data=X_test_scaled, index=[X_test.index], columns=[X_test.columns]) # turn created np.array back to pandas df


In [39]:
# Grid search in cross validation setting

# Create a 20 fold stratified CV iterator
#cv_20fold = model_selection.StratifiedKFold(n_splits=10)
results = []
best_n_trees = []

# Loop over the folds
#for train_index, validation_index in cv_20fold.split(X, Y): #split train data in training and validation data

    # Split the data properly
    #X_train = X[train_index]
    #y_train = Y[train_index]
    
    #X_validation = X[validation_index]
    #y_validation = Y[validation_index]

# Create a grid search to find the optimal k using a gridsearch and 10-fold cross validation
parameters = {"n_estimators": list(range(1, 26, 2))} # range moet gekozen worden 
rfc = RandomForestClassifier() 
# cv_10fold = model_selection.StratifiedKFold(n_splits=10) # folds moet gekozen worden en stratification
grid_search = GridSearchCV(rfc, parameters, scoring='roc_auc') #scoring moet gekozen worden
grid_search.fit(X_train_scaled, y_train)

# Get resulting classifier
clf = grid_search.best_estimator_
print(f'Best classifier: k={clf.n_estimators}')
#best_n_trees.append(clf.n_estimators)
print(best_n_trees)
#results.append(grid_search.cv_results_) #mean_test_score
#print(results)
score= grid_search.best_score_
print(score)

# Test the classifier on the test data
#probabilities = clf.predict_proba(X_validation)
#scores = probabilities[:, 1]
      
# Get the auc
#auc_validation = metrics.roc_auc_score(y_validation, scores)
#results.append({
#'auc': auc_validation, 
#'k': clf.n_estimators,
#'set': 'validation'
      #})

# Create results dataframe and plot it
#results = pd.DataFrame(results)
#seaborn.boxplot(y='auc', x='set', data=results)

#optimal_n = int(np.median(best_n_trees)) # of median
#print(f"The optimal N={optimal_n}")

Best classifier: k=5
[]
0.5445946716551687


In [45]:
# Use the optimal parameters without any tuning to validate the optimal classifier
#clf = RandomForestClassifier(n_estimators=optimal_n)
#clf.fit(X, Y)
#y_pred = clf.predict(X)
#t = ("Misclassified: %d / %d" % ((Y != y_pred).sum(), X.shape[0]))

# test data 
y_true, y_pred = y_test, clf.predict(X_test_scaled)
auc=metrics.roc_auc_score(y_true, y_pred)
print(f'AUC:{auc}')
accuracy=metrics.accuracy_score(y_true, y_pred)
print(f'accuracy:{auc}')
F1=metrics.f1_score(y_true,y_pred)
print(f'F1:{auc}')
precision=metrics.precision_score(y_true,y_pred)
print(f'precision:{auc}')
recall=metrics.recall_score(y_true, y_pred)
print(f'recall:{auc}')

AUC:0.42246240601503754
accuracy:0.42246240601503754
F1:0.42246240601503754
precision:0.42246240601503754
recall:0.42246240601503754


In [0]:
# scoring classifier   
    auc=metrics.roc_auc_score(Y_r, y_score)
    accuracy=metrics.accuracy_score(Y_r, y_pred)
    F1=metrics.f1_score(Y_r,y_pred)
    precision=metrics.precision_score(Y_r,y_pred)
    recall=metrics.recall_score(Y_r, y_pred)